In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
%matplotlib inline
from dataScript import getDataset
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

#data = pd.read_csv('csv_result-aahaber-smooth_Corpus.csv', encoding='iso-8859-9')
data = getDataset()
# data = data.iloc[:,0].values
# features = []
# labels = []
# for d in data:
#     words = d.split(';')
#     features.append(words[2])
#     labels.append(int(words[1])-1)
features = data.iloc[:,0].values
labels = data.iloc[:,1].values
lenth = 0
for sen in features:
    words = word_tokenize(sen)
    if len(words) > lenth:
        lenth = len(words)

print(lenth)
f = open("stop_words.txt", "r", encoding='utf8')
stop = []
for x in f:
  stop.append(x.replace('\n',''))
f.close()

processed_features = []

for sentence in range(0, len(features)):
    processed_feature = re.sub(r'\s\'\s', '\'', str(features[sentence]))
    processed_feature= re.sub(r'[.!?,\"\[\]]', '', processed_feature)
    
    processed_feature= re.sub(r'(?<=\d)(\'\s)', '\'', processed_feature)    
    processed_feature = processed_feature.lower()
    processed_feature= re.sub(r'ü', 'u', processed_feature)
    processed_feature= re.sub(r'ö', 'o', processed_feature)
    processed_feature= re.sub(r'ı', 'i', processed_feature)
    processed_feature= re.sub(r'ğ', 'g', processed_feature)
    #processed_feature= re.sub(r'ş', 's', processed_feature)
    processed_feature= re.sub(r'ç', 'c', processed_feature)
    # Replaces #hashtag with hashtag
    processed_feature = re.sub(r'#(\S+)', r' \1 ', processed_feature)
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    processed_feature = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', processed_feature)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    processed_feature = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', processed_feature)
    # Love -- <3, :*
    processed_feature = re.sub(r'(<3|:\*)', ' EMO_POS ', processed_feature)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    processed_feature = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', processed_feature)
    # Sad -- :-(, : (, :(, ):, )-:
    processed_feature = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', processed_feature)
    # Cry -- :,(, :'(, :"(
    processed_feature = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', processed_feature)

    processed_feature = re.sub(r'(.)\1+', r'\1\1', processed_feature)
    tknzr = TweetTokenizer()
    words = tknzr.tokenize(processed_feature)
    newWords = [w for w in words if not w in stop]
    processed_features.append(str(newWords))


#print(processed_features[10:])
# all_words = []
# tknzr = TweetTokenizer()
# for sen in processed_features:
#     words = tknzr.tokenize(sen)    
#     #print(words)
#     filtered_words = [w for w in words if not w in stop]
#     all_words.append(str(filtered_words))
#     #print(filtered_words)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=1)
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

#print(X_train)


vocab_size = len(tokenizer.word_index) + 1


X_train = pad_sequences(X_train, padding='post', maxlen=lenth)
X_test = pad_sequences(X_test, padding='post', maxlen=lenth)

#print(X_train)


#print(y_train)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=lenth))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=20, batch_size=120, verbose=1)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))




60, 2, 1, 60, 858, 30, 124, 46, 367, 1082, 64], [1, 1, 8, 1, 1, 2, 21, 1, 1, 14, 18, 19, 1091, 1, 5], [36, 116, 22, 253, 37, 868, 37, 566, 2, 3, 26], [118, 2, 29, 439], [1, 1, 2], [41, 308, 34, 124, 46, 17, 24, 163, 524, 23, 135], [8, 1200, 127, 65, 234, 9, 100, 507], [570, 946, 56, 109, 321, 918, 12], [2, 3, 4], [6, 193, 696, 1370, 2, 1286, 246, 1127, 1052], [101, 254, 1268, 14, 66, 314, 910, 99, 664, 314, 840, 99, 1269, 204, 1270, 20, 665, 911], [259, 2, 677, 135, 1809, 977, 173, 259, 259, 743, 89, 8], [119, 221, 8, 2, 1259, 336, 25, 111, 689, 230, 345, 62], [857, 791, 2, 31, 191, 78, 577, 1080, 1, 5, 1, 5, 1, 5], [57, 31, 1637, 1, 5, 535, 6, 1381, 159], [667, 2, 631, 36, 420, 59, 631, 82, 100, 232], [556, 145, 908, 130, 2, 9], [55, 417, 142, 11, 13, 7, 1110, 38, 762, 1, 241], [2, 640, 3, 4, 1, 1, 1, 1], [63], [159, 1374, 2, 923], [1454, 170, 22, 61, 43, 79, 356, 652, 1034, 120], [2, 53, 3, 4, 443, 8, 3, 4], [9, 505, 296], [119, 515, 565, 849, 12, 1, 5, 2, 46, 3, 4], [34, 1779, 51, 5